#### * _We used Pytorch for our CNN models_ *

# __Why Detect Lane Markings__
For our project, we are interested in working on lane detection for autonomous vehicles using lane markings painted on the roads. The task of lane detection is still vital to the success of autonomous vehicles because it is pertinent that an autonomous vehicle can exactly recognize which lane it is in and where it needs to go. If it is not able to do that, the vehicles might be going in the middle of two lanes which can be dangerous for other vehicles in the vicinity. Also, lane detection is inevitable to perform maneuvers like lane changes and also for predicting other vehicles’ behavior.

# __Build a lane extractor from an image__
In this project, we tried to train a Convolutional Neural Network (CNN) to extract lane markings as features from given images of roads with lanes. 

### __Dataset Description__
We plan on using the __CULane Dataset__ (Chinese University Lane Dataset) which contains 88,880 images for the training set, 9,675 images for the validation set and 34,680 images for the test set. Furthermore, the test set is divided into one normal and 8 challenging categories that tests the network on harder images. The dataset is desirable because it contains images from a variety of settings like crowded, night, no line, shadow, curve, etc. Such diversity in the dataset is desirable for training the model.	

### __Modelling Pipeline__
  1. Extract the images from the dataset stored on Drive
  2. Organize the images into raw images and labelled value
  3. Train the Model
  4. Get a reasonable validation accuracy
  5. Test the Model on unseen images 

In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from os.path import join
import os

ROOT = '/content/drive'     # default for the drive
CODE = 'My Drive/cs231n/Project/LaneNet'       # path to our code on Drive
DATASET = 'My Drive/cs231n/Project/dataset/trainset'

CODE_PATH = join(ROOT, CODE)
DATASET_PATH = join(ROOT, DATASET)

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('{}'.format(CODE_PATH))



Mounted at /content/drive


## Some Imports


In [0]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

### Let's visualize some of the images we have
We have a contiguous set of 20 images frame-by-frame for each scene, with the 20th frame labelled in our dataset. <br>
Let's visualize these images to understand what each scene information contains:


In [35]:
# Obtaining a list of paths for each image
"""
FIXED_PATH = DATASET_PATH + '/clips/0313-1/60/'
items = os.listdir(FIXED_PATH)
n_images = len(items)
plt_idx = 0
fig = plt.figure()
n_cols = 20
for image in items: 
  plt_idx += 1
  full_path = FIXED_PATH + image
  a = fig.add_subplot(n_cols, np.ceil(n_images/float(n_cols)), plt_idx)
  img = plt.imread(full_path)
  #plt.imshow(img.astype('uint8'))
fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
plt.show()
"""

"\nFIXED_PATH = DATASET_PATH + '/clips/0313-1/60/'\nitems = os.listdir(FIXED_PATH)\nn_images = len(items)\nplt_idx = 0\nfig = plt.figure()\nn_cols = 20\nfor image in items: \n  plt_idx += 1\n  full_path = FIXED_PATH + image\n  a = fig.add_subplot(n_cols, np.ceil(n_images/float(n_cols)), plt_idx)\n  img = plt.imread(full_path)\n  #plt.imshow(img.astype('uint8'))\nfig.set_size_inches(np.array(fig.get_size_inches()) * n_images)\nplt.show()\n"

# __1. Extract the images from the dataset stored on Drive__ 

We have already done so in our Operational.ipynb notebook

# __2. Organize the images into raw images and labelled value__

Here, we will generate labelled data for our train, validation and test splits. We will generate labelled data as bindary masks since the original dataset doesn't come preloaded with this information.
<br>
Moreover, instead of loading data manually, we decided on using Pytorch to create a custom Dataset and DataLoader
<br> 
More details on the DataLoader setup can be found in `LaneNet/Data/Tusimple.py`
<br>

## Some imports for the modules

In [0]:
import argparse
import json
import os
import shutil
import time

import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

#from config import *
import data
from model import SCNN
from utils.tensorboard import TensorBoard
from utils.transforms import *
from utils.lr_scheduler import PolyLR

## Loading a pretrained model if it exists